# Geração de Databases com SQLalchemy e PosgreSQL

O presente notebook demonstra a construção de bancos de dados (BD) SQL locais a partir do uso das ferramentas SQLalchemy e PostgreSQL, para fins de realização de uma análise exploratória de dados descritos em arquivos CSV. Para tal, fizemos uso do dataset [Covid-19 Data Sharing](https://agencia.fapesp.br/covid-19-data-sharingbr-makes-more-datasets-available/35348) disponibilizado pela Agência FAPESP.

## Autores

| Nome | nUSP |
| :--- | :--- |
| Guilherme de Abreu Barreto | 12543033 |
| Lucas Eduardo Gulka Pulcinelli | 12547336 |
| Vinicio Yusuke Hayashibara | 13642797 |

## Configuração

É necessário o ao correto funcionamento deste projeto possuir uma instalação local de PostgreSQL e atribuir os valores correspondentes para acesso a este nas seguintes constantes:

- `DATABASE`: O nome do database onde serão carregadas as informações. Atente-se se este não corresponde ao nome de um database preexistente **ou que esteja sendo acessado**, pois este será então sobrescrito.

- `USER` e `PASSWORD`: Informações de autententicação válidas e com privilégios para a criação de bancos de dados no servidor.

- `HOST` e `PORT`: A URL e porta para realização do acesso ao servidor.

- `BATCH_SIZE`: O número máximo de operações sobre o BD a serem realizadas conjuntamente. Recomenda-se ser em um número o qual caiba na memória RAM que você dispõe. O número abaixo foi capaz de caber confortávelmente em 10 GiB de RAM **na minha máquina**. 

In [1]:
DEFAULT_DATABASE = "postgres"
FAPCOV_DATABASE = "fapcov2103"
USER = "postgres"
PASSWORD = "postgres"
HOST = "localhost"
PORT = 5432
BATCH_SIZE = 2 * 10**6

DATABASE_URI = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}/{FAPCOV_DATABASE}"

## Carregamento das dependências deste projeto

In [2]:
!pip install psycopg2-binary sqlalchemy[postgres] tqdm pandas

import enum
import pandas as pd
import re
from datetime import datetime, date
from psycopg2.errors import UniqueViolation
from sqlalchemy import (
    CheckConstraint as constraint,
    Enum,
    Date,
    ForeignKey as fk,
    String,
    MetaData,
    Table,
    TypeDecorator,
    create_engine,
    column as sql_column,
    insert,
    text,
    PrimaryKeyConstraint as pkc
)
from sqlalchemy.orm import (
    Mapped,
    Session,
    declarative_base,
    relationship,
    sessionmaker,
    mapped_column as column,
    validates,
)
from pathlib import Path
from tqdm import tqdm
from typing import Any, Annotated, final
from dateutil.parser import parse

## Funções e tipos de dados auxiliares

Algumas funções e tipos de dados os quais utilizamos em nossa implementação, mas cuja funcionalidade provavelmente não será crucial ao caso geral.

In [3]:
def backref(back_populates: str) -> Mapped[Any]:
    return relationship(back_populates=back_populates)


def childOf(back_populates: str) -> Mapped[Any]:
    return relationship(
        back_populates=back_populates,
        cascade="all, delete-orphan",
    )


def normalize_column_name(column_name: str) -> str:
    return column_name.lower()

def parse_line(v):
    try:
        return parse(v)
    except:
        return None

def parse_date(date_column: pd.Series) -> pd.Series:
    return  date_column.apply(parse_line)


estados = [
    "AC",  # Acre
    "AL",  # Alagoas
    "AP",  # Amapá
    "AM",  # Amazonas
    "BA",  # Bahia
    "CE",  # Ceará
    "DF",  # Distrito Federal
    "ES",  # Espírito Santo
    "GO",  # Goiás
    "MA",  # Maranhão
    "MT",  # Mato Grosso
    "MS",  # Mato Grosso do Sul
    "MG",  # Minas Gerais
    "PA",  # Pará
    "PB",  # Paraíba
    "PR",  # Paraná
    "PE",  # Pernambuco
    "PI",  # Piauí
    "RJ",  # Rio de Janeiro
    "RN",  # Rio Grande do Norte
    "RS",  # Rio Grande do Sul
    "RO",  # Rondônia
    "RR",  # Roraima
    "SC",  # Santa Catarina
    "SP",  # São Paulo
    "SE",  # Sergipe
    "TO",  # Tocantins
]

## Definição das tabelas comuns

Abaixo descrevemos a estrutura pretendida às tabelas Pacientes, ExamLabs e Despachos, comuns a todos o BDs.

In [4]:
Base = declarative_base()

class PacienteBase(Base):
    __abstract__: bool = True

    ic_sexo: Mapped[str] = column(
        Enum('M', 'F', name='sexo_enum'),
        comment="Sexo do Paciente. F - Feminino; M - Masculino"
    )
    aa_nascimento: Mapped[str | None] = column(
        String(4),
        comment="Ano de nascimento do Paciente. 4 caracteres alfanuméricos. Os 4 dígitos do ano do nascimento; ou AAAA - para ano de nascimento igual ou anterior a 1930 (visando anonimização); YYYY - quaisquer outros anos, em caso de anonimização do ano"
    )
    cd_pais: Mapped[str | None] = column(
        String(2),
        comment="Pais de residencia do Paciente. 2 caracteres alfanuméricos. BR ou XX (país estrangeiro)"
    )
    cd_uf: Mapped[str | None] = column(
        Enum(*estados, name='estado_enum'),
        comment="Unidade da Federacao de residencia do Paciente. 2 caracteres alfanuméricos"
    )
    cd_municipio: Mapped[str | None] = column(
        comment="Municipio de residencia do Paciente. Alfanumérico."
    )
    cd_cepreduzido: Mapped[str | None] = column(comment="[Descrição não encontrada nos comentários]")

    @validates("aa_nascimento")
    def validates_nascimento(self, _key: str, value: str) -> str:
        match value:
            case "AAAA" | "YYYY":
                return value
            case year if len(year) == 4 and year.isdigit():
                return year
            case _:
                raise ValueError(
                    f"Invalid AA_Nascimento value: {value}. Must be 'AAAA', 'YYYY', or a 4-digit number"
                )


class Paciente(PacienteBase):
    """
    Tabela de pacientes Covid-19 FAPESP
    """

    __tablename__: str = "pacientes"

    id_paciente: Mapped[str] = column(
        primary_key=True,
        comment="Identificação única do paciente (correlaciona com o ID_PACIENTE de todos os arquivos onde aparece). 32 caracteres alfanuméricos"
    )

    # Relações
    exames: Mapped[list["ExamLab"]] = childOf("paciente")
    desfechos: Mapped[list["Desfecho"]] = childOf("paciente")



class ExamLab(Base):
    """
    Tabela de exames Covid-19 FAPESP
    """

    __tablename__: str = "examlabs"

    id: Mapped[int] = column(autoincrement=True, primary_key=True)
    id_paciente: Mapped[str] = column(
        fk("pacientes.id_paciente"),
        comment="Identificação única do paciente (correlaciona com o ID_PACIENTE de todos os arquivos onde aparece). 32 caracteres alfanuméricos"
    )
    id_atendimento: Mapped[str | None] = column(
        comment="Identificação única do atendimento. Correlaciona com o ID_ATENDIMENTO de todas as tabelas onde aparece. 32 caracteres alfanuméricos"
    )
    de_exame: Mapped[str | None] = column(
        comment="Descrição do exame realizado. Alfanumérico. Exemplo: HEMOGRAMA, sangue total / GLICOSE, plasma / SODIO, soro / POTASSIO, soro. Um exame é composto por 1 ou mais analitos."
    )
    de_resultado: Mapped[str | None] = column(
        comment="Resultado do exame, associado ao DE_ANALITO. Alfanumérico. Se DE_ANALITO exige valor numérico, NNNN se inteiro ou NNNN,NNN se casas decimais; Se DE_ANALITO exige qualitativo, String com domínio restrito; Se DE_ANALITO por observação microscópica, String conteúdo livre. Exemplo de dominio restrito - Positivo, Detectado, Reagente, nâo reagente, etc. Exemplo de conteúdo livre - 'não foram observados caracteres tóxico-degenerativos nos neutrófilos, não foram observadas atipias linfocitárias'"
    )
    dt_coleta: Mapped[date | None] = column(
        comment="Data em que o material foi coletado do paciente"
    )
    de_origem: Mapped[str | None] = column(
        comment="Local de Coleta do exame. 4 caracteres alfabéticos: LAB – Exame realizado por paciente em uma unidade de atendimento laboratorial; HOSP – Exame realizado por paciente dentro de uma Unidade Hospitalar; UTI - exame realizado na UTI"
    )
    de_analito: Mapped[str | None] = column(
        comment="Descrição do analito. Alfanumérico. Exemplo: Eritrócitos / Leucócitos / Glicose / Ureia / Creatinina. Para o exame Hemograma, tem o resultado de vários analitos: Eritrócitos, Hemoglobina, Leucócitos, Linfócitos, etc. A maioria dos exames tem somente 1 analito, por exemplo Glicose, Colesterol Total, Uréia e Creatinina."
    )
    cd_unidade: Mapped[str | None] = column(
        comment="Unidade de Medida utilizada na Metodologia do laboratório específico para analisar o exame. Alfanumérico. Exemplo: g/dL (gramas por decilitro)"
    )
    de_valor_referencia: Mapped[str | None] = column(
        comment="Faixa de valores de referência. Alfanumérico. Resultado ou faixa de resultados considerado normal para este analito. Exemplo para Glicose: 75 a 99"
    )

    @property
    def de_resultnum(self) -> float | None:
        """
        Extrai valor numérico do resultado ou atribui códigos especiais para resultados textuais.
        Baseado na lógica do script COVID19_Corrige_21_02.sql
        """
        if not self.de_resultado:
            return None

        # Extrai valor numérico do resultado
        numeric_match = re.search(r"-?\d+[,.]?\d*", self.de_resultado)
        if numeric_match:
            numeric_str = numeric_match.group().replace(",", ".")
            try:
                return float(numeric_str)
            except ValueError:
                pass

        # Aplica códigos especiais para exames COVID
        if self.de_exame and re.search(
            r"(covid)|(sars.cov.2)|(corona)", self.de_exame, re.IGNORECASE
        ):
            resultado_lower = self.de_resultado.lower()

            if re.search(r"detectados anticorpos", resultado_lower):
                return -1000.0
            elif re.search(
                r"(n.o detectado)|(n.o reagente)|(negativo)|(aus.ncia de anticorpos)",
                resultado_lower,
            ):
                return -1111.0
            elif re.search(r"(detectado)|(reagente)|(positivo)", resultado_lower):
                return -1000.0
            elif re.search(r"(indetect.avel)|(inconclusivo)", resultado_lower):
                return -1234.0
            else:
                return -2222.0

        return None

    # Relações
    paciente: Mapped["Paciente"] = backref("exames")


class Desfecho(Base):
    """
    Tabela de desfechos Covid-19 FAPESP
    """

    __tablename__: str = "desfechos"

    id_paciente: Mapped[str] = column(
        fk("pacientes.id_paciente"),
        comment="Identificação única do paciente (correlaciona com o ID_PACIENTE de todos os arquivos onde aparece. 32 caracteres alfanuméricos)"
    )
    id_atendimento: Mapped[str] = column(
        comment="Identificação única do atendimento. Cada atendimento tem um desfecho. Correlaciona com ID_ATENDIMENTO de todas as tabelas onde aparece"
    )
    dt_atendimento: Mapped[date | None] = column(
        comment="Data de realização do atendimento"
    )
    de_tipo_atendimento: Mapped[str] = column(
        comment="Descrição do tipo de atendimento realizado. Texto livre. Exemplo: Pronto atendimento."
    )
    id_clinica: Mapped[int] = column(
        comment="Identificação da clínica onde o evento aconteceu. Numérico. Exemplo: 1013"
    )
    de_clinica: Mapped[str] = column(
        comment="Descrição da clínica onde o evento aconteceu. Texto livre. Exemplo: Cardiologia"
    )
    dt_desfecho: Mapped[date | None] = column(
        comment="Data do desfecho - Nulo se DE_DESFECHO for óbito"
    )
    de_desfecho: Mapped[str] = column(
        comment="Descriçao do desfecho. Texto livre. Exemplo: Alta médica melhorado"
    )

    # Relações
    paciente: Mapped["Paciente"] = backref("desfechos")

    __table_args__: tuple[pkc,] = (pkc("id_paciente", "id_atendimento"),)

## População dos bancos de dados

Abaixo descrevemos a lógica para população dos bancos de dados. Os datasets que descrevem a cada tabela de cada banco de dados estão localizados em uma pasta `dataset` colocada no mesmo diretório que este notebook.

```bash
~/Public/USP/Ciência da Computação/Semestre 6/Mineração de dados/01-Introdução-Preparação de dados main*
❄️impure ❯ exa --tree
.
├── datasets
│   ├── BPSP
│   │   ├── BPSP_Desfechos.csv
│   │   ├── BPSP_ExamLabs.csv
│   │   └── BPSP_Pacientes.csv
│   ├── Einstein
│   │   ├── Einstein_ExamLabs.csv
│   │   └── Einstein_Pacientes.csv
│   ├── GrupoFleury
│   │   ├── GrupoFleury_ExamLabs.csv
│   │   └── GrupoFleury_Pacientes.csv
│   ├── HC
│   │   ├── HC_ExamLabs.csv
│   │   └── HC_Pacientes.csv
│   └── HSL
│       ├── HSL_Desfechos.csv
│       ├── HSL_ExamLabs.csv
│       └── HSL_Pacientes.csv
└── 'Geração de Databases com SQLalchemy e PostgreSQL.ipynb'
```
Como se vê, os datasets encontram-se nomeados de maneira padronizada, e os nomes das colunas de suas tabelas correspondem aos nomes dados aos atributos das classes que aqui descrevem as tabelas contidas nos BDs.

In [5]:
datasets_folder = "datasets"
hospitals = ["BPSP", "Einstein", "GrupoFleury", "HC", "HSL"]
tables_dict = {"Pacientes": Paciente, "ExamLabs": ExamLab, "Desfechos": Desfecho}

### Criação do Banco de Dados

As duas células seguintes executam a criação do banco de dados e dos _schemas_ para cada hospital, respectivamente.

In [ ]:
engine = create_engine(
    f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}/{DEFAULT_DATABASE}",
    echo=True
)

with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
    try:
        conn.execute(text(
        f"""
        SELECT pg_terminate_backend(pid)
        FROM pg_stat_activity
        WHERE datname = '{FAPCOV_DATABASE}';
        """
    ))
    except ProgrammingError as e:
        pass # Could not terminate connections (there are no connections)
    # NOTE: DROP DATABASE cannot run inside a transaction block, that is why we're
    # running it separately below.
    conn.execute(text(f"DROP DATABASE IF EXISTS {FAPCOV_DATABASE};"))
    conn.execute(text(f"CREATE DATABASE {FAPCOV_DATABASE};"))

engine = create_engine(DATABASE_URI, echo=True)

with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as conn:
    for hospital in hospitals + ["D2"]:
        conn.execute(text(f"CREATE SCHEMA {hospital}"))

In [7]:
def batch_insert(session: Session, hospital: str, table_name: str, table_class) -> None:
    chunks = pd.read_csv(
        f"{datasets_folder}/{hospital}/{hospital}_{table_name}.csv",
        delimiter="|",
        encoding="utf-8",
        low_memory=False,
        chunksize=BATCH_SIZE,
        dtype = {'CD_Unidade': str}
    )
    pbar = tqdm(
        chunks, desc=f"Populating table {table_name} from {hospital} schema"
    )

    if table_name == "Pacientes":
        for df in pbar:
            df.rename(columns=normalize_column_name, inplace=True)
            # AA_Nascimento verification. Condition 1: Value is 'AAAA' or 'YYYY'
            is_placeholder = df['aa_nascimento'].isin(['AAAA', 'YYYY'])
    
            # Condition 2: Value is a 4-digit string
            # Ensure it's a string before using .str accessor
            is_4_digit_year = (df['aa_nascimento'].astype(str).str.isdigit()) & \
                              (df['aa_nascimento'].astype(str).str.len() == 4)
    
            # Combine conditions: A row is valid if it meets Condition 1 OR Condition 2
            valid_mask = is_placeholder | is_4_digit_year
    
            df.loc[~valid_mask, 'aa_nascimento'] = None
            df['cd_pais'] = df['cd_pais'].replace('XX', None)
            df['cd_uf'] = df['cd_uf'].replace('UU', None)
            df['cd_municipio'] = df['cd_municipio'].replace('MMMM', None)
            df['cd_cepreduzido'] = df['cd_cepreduzido'].replace('CCCC', None)
            session.execute(insert(table_class), df.to_dict('records'))
    else:
        result = session.execute(text('SELECT ID_Paciente FROM Pacientes'))
        valid_patient_ids = {row[0] for row in result}
        if table_name == "ExamLabs":
            for df in pbar:
                df.rename(columns=normalize_column_name, inplace=True)
                df = df[df['id_paciente'].isin(valid_patient_ids)].copy()
                df['dt_coleta'] = parse_date(df['dt_coleta'])
                df = df.astype(object).where(pd.notna(df), None)
                session.execute(insert(table_class), df.to_dict('records'))
        else:
            for df in pbar:
                df.rename(columns=normalize_column_name, inplace=True)
                df = df[df['id_paciente'].isin(valid_patient_ids)].copy()
                df['dt_atendimento'] = parse_date(df['dt_atendimento'])
                df['dt_desfecho'] = parse_date(df['dt_desfecho'])
                df = df.astype(object).where(pd.notna(df), None)
                session.execute(insert(table_class), df.to_dict('records'))


for hospital in hospitals:
    engine = create_engine(
        f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}/{FAPCOV_DATABASE}",
        connect_args={'options': f'-c search_path={hospital}'},
    )

    for table_name, table_class in tables_dict.items():
        Session = sessionmaker(bind=engine)
        with Session() as session:
            Base.metadata.create_all(engine)
            session.commit()

            try:
                batch_insert(session, hospital, table_name, table_class)
                session.commit()
            except FileNotFoundError:
                pass
            except Exception as e:
                print(e)


Populating table Pacientes from BPSP schema: 1it [00:08,  8.52s/it]
Populating table ExamLabs from BPSP schema: 4it [34:49, 522.44s/it]
Populating table Desfechos from BPSP schema: 1it [00:53, 53.07s/it]
Populating table Pacientes from Einstein schema: 1it [00:27, 27.39s/it]
Populating table ExamLabs from Einstein schema: 2it [26:33, 796.88s/it]
Populating table Pacientes from GrupoFleury schema: 1it [02:01, 121.77s/it]
Populating table ExamLabs from GrupoFleury schema: 10it [1:47:48, 646.81s/it]
Populating table Pacientes from HC schema: 1it [00:00,  1.16it/s]
Populating table ExamLabs from HC schema: 2it [09:47, 293.52s/it]
Populating table Pacientes from HSL schema: 1it [00:02,  2.10s/it]
Populating table ExamLabs from HSL schema: 1it [04:02, 242.32s/it]
Populating table Desfechos from HSL schema: 1it [00:09,  9.48s/it]


O resultado esperado desta execução é a criação dos seguintes BDs estruturados tal qual exibe os seguinte diagrama gerado usando a ferramenta DBeaver:

![Estrutura do BD, onde Pacientes figura como uma tabela associada a ExamLabs e Despachos](imgs/db_structure.png)

# Criação de novo banco de dados para a análise de dados

Em seguida, criamos um novo BD para conter dados agregados a todos os demais BDs. Isto, conforme os critérios de seleção vistos na tabela AnalisesCovid, descrita a seguir.

In [5]:
class AnaliseCovid(PacienteBase):
    """
    Tabela de análises Covid-19 FAPESP

    Complementa a tabela Paciente com dados relevantes sobre atendimentos,
    exames e desfechos extraídos das tabelas ExamLabs e Desfechos para análise
    epidemiológica de casos associados ao COVID-19
    """

    __tablename__: str = "AnalisesCovid"
    id: Mapped[int] = column(autoincrement=True, primary_key=True)

    # Aggregated data
    id_paciente: Mapped[str] = column(
        comment="Identificação única do paciente (32 caracteres alfanuméricos)"
    )
    id_atendimento: Mapped[str | None] = column(
        comment="Identificação única do atendimento (32 caracteres alfanuméricos)"
    )
    dt_coleta: Mapped[date | None] = column(
        comment="Data em que o material foi coletado para exame"
    )
    dt_atendimento: Mapped[date | None] = column(
        comment="Data de realização do atendimento"
    )
    dt_desfecho: Mapped[date | None] = column(
        comment="Data do desfecho do paciente (alta, óbito, etc.)"
    )
    de_desfecho: Mapped[str | None] = column(
        comment="Descrição do desfecho do paciente (ex: 'Alta médica melhorado', 'Óbito')"
    )
    de_exame: Mapped[str | None] = column(
        comment="Descrição do exame realizado (ex: 'Teste COVID-19', 'Hemograma')"
    )
    de_resultado: Mapped[str | None]

    # Added metadata
    de_classe: Mapped[str | None] = column(
        Enum('P', 'N', name='classe_enum'),
        comment="Resultado do exame COVID-19 simplificado (P - Positivo, N - Negativo, None - Outro/Indeterminado)"
    )
    de_hospital: Mapped[str] = column(
        comment="Identificação do hospital de origem dos dados (BPSP, Einstein, GrupoFleury, HC, HSL)"
    )

class PacienteD2(Paciente):
    de_hospital: Mapped [str]

class ExamLabD2(ExamLab):
    de_hospital: Mapped [str]

class DesfechoD2(Desfecho):
    de_hospital: Mapped [str]

## População do BD D2 com dados dos demais BDs

Em seguida acessamos aos demais BDs um a um e criamos registros correspondentes no DB D2:

In [25]:
# Populate a new database D2 with its tables
engine = create_engine(
    DATABASE_URI,
    connect_args={'options': f'-c search_path=D2'},
    echo = True
)
Session = sessionmaker(bind=engine)

with Session() as session:
    tables = [
        AnaliseCovid.__table__,
        PacienteD2.__table__,
        ExamLabD2.__table__,
        DesfechoD2.__table__,
    ]
    Base.metadata.create_all(engine, tables=tables)
    session.commit()

2025-11-28 17:38:13,456 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-11-28 17:38:13,457 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-28 17:38:13,460 INFO sqlalchemy.engine.Engine select current_schema()
2025-11-28 17:38:13,461 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-28 17:38:13,463 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-11-28 17:38:13,464 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-28 17:38:13,467 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-28 17:38:13,473 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [26]:
engine = create_engine(DATABASE_URI)

with engine.connect() as conn:
    conn.execution_options(isolation_level="AUTOCOMMIT")

    for table_name, table_class in tables_dict.items():
        for hospital in hospitals:
            print(f"--- Copying {hospital}.{table_name} data ---")    
            columns = [
                col for col in table_class.__table__.columns if col.name != "id" 
            ]
            insert_columns = ", ".join([f'{col.name}' for col in columns])

            # Apply the explicit DOUBLE CAST for Enum types
            select_columns = ", ".join([
                f'{col.name}::text::d2.{col.type.name}'
                if isinstance(col.type, Enum)
                else f'"{col.name}"'
                for col in columns
                if col.name != "de_hospital"
            ])
            
            copy_sql = f"""
                INSERT INTO d2.{table_name.lower()} ({insert_columns})
                SELECT {select_columns}, '{hospital}' as de_hospital 
                FROM {hospital}.{table_name.lower()}
            """
            try:
                conn.execute(text(copy_sql))
                print(f"Copied {hospital}.{table_name} to d2.{table_name}")
            except Exception as e:
                print(f"Error copying {hospital}.{table_name}: {e}")


--- Copying BPSP.Pacientes data ---
Copied BPSP.Pacientes to d2.Pacientes
--- Copying Einstein.Pacientes data ---
Copied Einstein.Pacientes to d2.Pacientes
--- Copying GrupoFleury.Pacientes data ---
Copied GrupoFleury.Pacientes to d2.Pacientes
--- Copying HC.Pacientes data ---
Copied HC.Pacientes to d2.Pacientes
--- Copying HSL.Pacientes data ---
Copied HSL.Pacientes to d2.Pacientes
--- Copying BPSP.ExamLabs data ---
Copied BPSP.ExamLabs to d2.ExamLabs
--- Copying Einstein.ExamLabs data ---
Copied Einstein.ExamLabs to d2.ExamLabs
--- Copying GrupoFleury.ExamLabs data ---
Copied GrupoFleury.ExamLabs to d2.ExamLabs
--- Copying HC.ExamLabs data ---
Copied HC.ExamLabs to d2.ExamLabs
--- Copying HSL.ExamLabs data ---
Copied HSL.ExamLabs to d2.ExamLabs
--- Copying BPSP.Desfechos data ---
Copied BPSP.Desfechos to d2.Desfechos
--- Copying Einstein.Desfechos data ---
Copied Einstein.Desfechos to d2.Desfechos
--- Copying GrupoFleury.Desfechos data ---
Copied GrupoFleury.Desfechos to d2.Desfechos

In [ ]:
engine = create_engine(
    DATABASE_URI,
    connect_args={'options': f'-c search_path=D2'},
)
Session = sessionmaker(bind=engine)

with Session() as session:
    query = (
        session.query(
            PacienteD2,          # Select the entire PacienteD2 object
            ExamLabD2.de_exame,
            ExamLabD2.de_resultado,
            ExamLabD2.dt_coleta,
            ExamLabD2.id_atendimento,
            DesfechoD2.de_desfecho,
            DesfechoD2.dt_desfecho,
            DesfechoD2.dt_atendimento
        )
        .select_from(PacienteD2)
        .join(ExamLabD2, PacienteD2.id_paciente == ExamLabD2.id_paciente)
        .join(
            DesfechoD2,
            (PacienteD2.id_paciente == DesfechoD2.id_paciente)
            & (ExamLabD2.id_atendimento == DesfechoD2.id_atendimento),
        )
        .filter(
            ExamLabD2.de_exame.ilike("%covid%")
            | ExamLabD2.de_exame.ilike("%corona%")
            | ExamLabD2.de_exame.ilike("%sars%")
        )
    )

    records = []
    for chunk in tqdm(query.yield_per(BATCH_SIZE), desc="Processando registros"):
        temp_exam = ExamLab(
            de_exame=chunk.de_exame, de_resultado=chunk.de_resultado
        )

        match temp_exam.de_resultnum:
            case -1000:
                classe = "P"
            case -1111:
                classe = "N"
            case _:
                classe = None

        records.append (
            {
                "id_paciente": chunk.PacienteD2.id_paciente, 
                "id_atendimento": chunk.id_atendimento,
                "ic_sexo": chunk.PacienteD2.ic_sexo,
                "aa_nascimento": chunk.PacienteD2.aa_nascimento,
                "cd_uf": chunk.PacienteD2.cd_uf,
                "cd_pais": chunk.PacienteD2.cd_pais,
                "cd_municipio": chunk.PacienteD2.cd_municipio,
                "cd_cepreduzido": chunk.PacienteD2.cd_cepreduzido,
                "dt_atendimento": chunk.dt_atendimento,
                "dt_coleta": chunk.dt_coleta,
                "de_exame": chunk.de_exame,
                "dt_desfecho": chunk.dt_desfecho,
                "de_desfecho": chunk.de_desfecho,
                "de_resultado": chunk.de_resultado,
                "de_classe": classe,
                "de_hospital": chunk.PacienteD2.de_hospital,
            }
        )
    if records:
        session.execute(insert(AnaliseCovid), records)
        session.commit()

df = pd.read_sql_table("AnalisesCovid", engine)
print("\n--- Verificando tabela resultante ---")
print(df.head())
print(f"\nTotal de entradas em AnalisesCovid: {len(df)}")

Processando registros: 0it [00:00, ?it/s]